In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
import operator

# Load environment variables from .env
load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.95,
    top_k=40,
    google_api_key=os.getenv("GEMINI_API_KEY") 
)

In [3]:
class QuadState(TypedDict):
    a: int
    b: int
    c: int

    equation: str
    discriminant: int
    result: str
    

In [4]:
def show_equation(state: QuadState) -> QuadState:
    a = state['a']
    b = state['b']
    c = state['c']
    equation = f"{a}x2 + {b}x + {c} = 0"
    state['equation'] = equation

    return {'equation': state['equation']}

def calculate_discriminant(state: QuadState) -> QuadState:
    a = state['a']
    b = state['b']
    c = state['c']
    discriminant = b**2 - 4*a*c
    state['discriminant'] = discriminant

    return {'discriminant': state['discriminant']}

def real_roots(state: QuadState) -> QuadState:
    root1 = (-state['b'] + (state['discriminant']**0.5)) / (2 * state['a'])
    root2 = (-state['b'] - (state['discriminant']**0.5)) / (2 * state['a'])

    result = f"The roots are: {root1} and {root2}"

    return {'result': result}

def repeated_root(state: QuadState) -> QuadState:
    root = -state['b'] / (2 * state['a'])
    result = f"The repeated root is: {root}"

    return {'result': result}

def complex_roots(state: QuadState) -> QuadState:
    result = "There are no real roots."

    return {'result': result}

def checkcondition(state: QuadState) -> QuadState:
    if state['discriminant'] > 0:
        return "real_roots"
    elif state['discriminant'] == 0:
        return "repeated_root"
    else:
        return "complex_roots"
    

In [5]:
graph = StateGraph(QuadState)

graph.add_node('show_equation', show_equation)
graph.add_node('calculate_discriminant', calculate_discriminant)
graph.add_node('real_roots', real_roots)
graph.add_node('repeated_root', repeated_root)
graph.add_node('complex_roots', complex_roots)

graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'calculate_discriminant')

graph.add_conditional_edges('calculate_discriminant',checkcondition)

workflow = graph.compile()

In [6]:
initial_state = {
    'a': 1,
    'b': 0,
    'c': 2
}

workflow.invoke(initial_state)

{'a': 1,
 'b': 0,
 'c': 2,
 'equation': '1x2 + 0x + 2 = 0',
 'discriminant': -8,
 'result': 'There are no real roots.'}